In [0]:
 dbutils.fs.mount(
     source='wasbs://demo1@datastorage123.blob.core.windows.net/',
     mount_point='/mnt/srini',
     extra_configs={
 'fs.azure.account.key.datastorage123.blob.core.windows.net': '6xEUxRWroFePxZjTYSsjxbm0OQXz4/T67SJ+36IQziH7RuvG1XY+JIK9QLskt7Y2Ovi4lRXp2BpY+AStiKJgPQ=='
    }
 )

In [0]:
spark.conf.set(
  "fs.azure.account.key.datastorage123.dfs.core.windows.net",
  "6xEUxRWroFePxZjTYSsjxbm0OQXz4/T67SJ+36IQziH7RuvG1XY+JIK9QLskt7Y2Ovi4lRXp2BpY+AStiKJgPQ=="
)
 
df = spark.read.csv("abfss://demo1@datastorage123.dfs.core.windows.net/Databrickssampledata.csv", header=True)
df.show()
 
 

+---+-------+----------+------+
| id|   name|department|salary|
+---+-------+----------+------+
|  1|  Alice|     Sales|  5000|
|  2|    Bob|     Sales|  6000|
|  3|Charlie|     Sales|  6000|
|  4|  David|        HR|  4000|
|  5|    Eve|        HR|  4000|
|  6|  Frank|        HR|  4500|
|  7|  Grace|        IT|  7500|
|  8|  Heidi|        IT|  7200|
|  9|   Ivan|        IT|  7200|
| 10|   Judy|   Finance|  6700|
| 11|  Kevin|   Finance|  6700|
| 12|  Laura|   Finance|  6200|
| 13|Mallory|     Sales|  5200|
| 14|   Neil|     Sales|  5200|
| 15|  Oscar|     Sales|  4800|
| 16|  Peggy|        HR|  4500|
| 17|Quentin|        HR|  4100|
| 18|   Ruth|   Finance|  5900|
| 19|  Steve|        IT|  7000|
| 20|  Trent|   Finance|  6300|
+---+-------+----------+------+
only showing top 20 rows



In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, rank, dense_rank, ntile

# Read the CSV with inferred schema
df = spark.read.option("header", True).option("inferSchema", True).csv(
    "abfss://demo1@datastorage123.dfs.core.windows.net/Databrickssampledata.csv"
)

# Define window specification
window_spec = Window.partitionBy("department").orderBy(df.salary.desc())

# Apply window functions
df_with_ranks = df.select(
    "*",
    row_number().over(window_spec).alias("row_num"),
    rank().over(window_spec).alias("rank"),
    dense_rank().over(window_spec).alias("dense_rank"),
    ntile(2).over(window_spec).alias("ntile_2")  # Split each department into 2 tiles
)

# Show the result
df_with_ranks.show()


+---+-------+----------+------+-------+----+----------+-------+
| id|   name|department|salary|row_num|rank|dense_rank|ntile_2|
+---+-------+----------+------+-------+----+----------+-------+
| 10|   Judy|   Finance|  6700|      1|   1|         1|      1|
| 11|  Kevin|   Finance|  6700|      2|   1|         1|      1|
| 20|  Trent|   Finance|  6300|      3|   3|         2|      1|
| 12|  Laura|   Finance|  6200|      4|   4|         3|      2|
| 18|   Ruth|   Finance|  5900|      5|   5|         4|      2|
|  6|  Frank|        HR|  4500|      1|   1|         1|      1|
| 16|  Peggy|        HR|  4500|      2|   1|         1|      1|
| 17|Quentin|        HR|  4100|      3|   3|         2|      1|
|  4|  David|        HR|  4000|      4|   4|         3|      2|
|  5|    Eve|        HR|  4000|      5|   4|         3|      2|
|  7|  Grace|        IT|  7500|      1|   1|         1|      1|
|  8|  Heidi|        IT|  7200|      2|   2|         2|      1|
|  9|   Ivan|        IT|  7200|      3| 

In [0]:
pivot_df = df.groupBy("name").pivot("department").sum("salary")
pivot_df.show()


+-------+-------+----+----+-----+
|   name|Finance|  HR|  IT|Sales|
+-------+-------+----+----+-----+
|  Grace|   null|null|7500| null|
|  Steve|   null|null|7000| null|
|  Trent|   6300|null|null| null|
|Quentin|   null|4100|null| null|
|  Heidi|   null|null|7200| null|
|Charlie|   null|null|null| 6000|
|   Judy|   6700|null|null| null|
|   Neil|   null|null|null| 5200|
|    Bob|   null|null|null| 6000|
|Mallory|   null|null|null| 5200|
|  Oscar|   null|null|null| 4800|
|  Alice|   null|null|null| 5000|
|    Eve|   null|4000|null| null|
|  Kevin|   6700|null|null| null|
|   Ruth|   5900|null|null| null|
|  David|   null|4000|null| null|
|    Uma|   null|null|null| 4900|
|   Ivan|   null|null|7200| null|
|  Peggy|   null|4500|null| null|
|  Frank|   null|4500|null| null|
+-------+-------+----+----+-----+
only showing top 20 rows



In [0]:
# Unpivot using stack()
unpivot_df = pivot_df.selectExpr(
    "name",
    "stack(3, 'HR', HR, 'Sales', Sales, 'IT', IT) as (department, salary)"
).where("salary is not null")

unpivot_df.show()


+-------+----------+------+
|   name|department|salary|
+-------+----------+------+
|  Grace|        IT|  7500|
|  Steve|        IT|  7000|
|Quentin|        HR|  4100|
|  Heidi|        IT|  7200|
|Charlie|     Sales|  6000|
|   Neil|     Sales|  5200|
|    Bob|     Sales|  6000|
|Mallory|     Sales|  5200|
|  Oscar|     Sales|  4800|
|  Alice|     Sales|  5000|
|    Eve|        HR|  4000|
|  David|        HR|  4000|
|    Uma|     Sales|  4900|
|   Ivan|        IT|  7200|
|  Peggy|        HR|  4500|
|  Frank|        HR|  4500|
+-------+----------+------+



In [0]:
from pyspark.sql.functions import explode
from pyspark.sql import Row

data = [Row(id=1, name="Alice", skills=["SQL", "Python", "Spark"])]
df = spark.createDataFrame(data)

df_exploded = df.select("id", "name", explode("skills").alias("skill"))
df_exploded.show()


+---+-----+------+
| id| name| skill|
+---+-----+------+
|  1|Alice|   SQL|
|  1|Alice|Python|
|  1|Alice| Spark|
+---+-----+------+



In [0]:
from pyspark.sql.functions import arrays_zip

data = [
    Row(name="Bob", subjects=["Math", "English"], scores=[90, 85])
]
df = spark.createDataFrame(data)

df_zipped = df.select("name", arrays_zip("subjects", "scores").alias("zipped"))
df_zipped.show(truncate=False)


+----+---------------------------+
|name|zipped                     |
+----+---------------------------+
|Bob |[{Math, 90}, {English, 85}]|
+----+---------------------------+



In [0]:
from pyspark.sql.functions import explode

df_flattened = df_zipped.select("name", explode("zipped").alias("subject_score"))
df_flattened.select(
    "name",
    "subject_score.subjects",
    "subject_score.scores"
).show()


+----+--------+------+
|name|subjects|scores|
+----+--------+------+
| Bob|    Math|    90|
| Bob| English|    85|
+----+--------+------+



In [0]:
df = spark.read.option("header", True).option("inferSchema", True).csv(
    "abfss://demo1@datastorage123.dfs.core.windows.net/Databrickssampledata.csv"
)
df_filtered = df.filter(df.salary > 6000)
df_filtered.explain()


== Physical Plan ==
*(1) Filter (isnotnull(salary#70) AND (salary#70 > 6000))
+- FileScan csv [id#67,name#68,department#69,salary#70] Batched: false, DataFilters: [isnotnull(salary#70), (salary#70 > 6000)], Format: CSV, Location: InMemoryFileIndex(1 paths)[abfss://demo1@datastorage123.dfs.core.windows.net/Databrickssampledata..., PartitionFilters: [], PushedFilters: [IsNotNull(salary), GreaterThan(salary,6000)], ReadSchema: struct<id:int,name:string,department:string,salary:int>




In [0]:
df_filtered = df.filter("salary > 5000").groupBy("department").avg("salary")
df_filtered.explain(True)


== Parsed Logical Plan ==
'Aggregate ['department], ['department, avg(salary#70) AS avg(salary)#81]
+- Filter (salary#70 > 5000)
   +- Relation [id#67,name#68,department#69,salary#70] csv

== Analyzed Logical Plan ==
department: string, avg(salary): double
Aggregate [department#69], [department#69, avg(salary#70) AS avg(salary)#81]
+- Filter (salary#70 > 5000)
   +- Relation [id#67,name#68,department#69,salary#70] csv

== Optimized Logical Plan ==
Aggregate [department#69], [department#69, avg(salary#70) AS avg(salary)#81]
+- Project [department#69, salary#70]
   +- Filter (isnotnull(salary#70) AND (salary#70 > 5000))
      +- Relation [id#67,name#68,department#69,salary#70] csv

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- HashAggregate(keys=[department#69], functions=[finalmerge_avg(merge sum#86, count#87L) AS avg(salary#70)#80], output=[department#69, avg(salary)#81])
   +- Exchange hashpartitioning(department#69, 200), ENSURE_REQUIREMENTS, [plan_id=71]
      +- HashAg

In [0]:
df_repartitioned = df.repartition(10)  # creates 10 partitions
df_repartitioned = df.repartition("department")


In [0]:
df_coalesced = df.coalesce(2)


In [0]:
df.write \
  .format("parquet") \
  .bucketBy(5, "department") \
  .sortBy("id") \
  .saveAsTable("bucketed_table")
